# [Day 11](https://adventofcode.com/2020/day/11): Seating System

In [1]:
import numpy as np

with open("../data/11.txt", "r") as f:
    seatingplan = np.array([list(l.strip()) for l in f.readlines()])

## Part 1

In [2]:
from itertools import product

def reseating(seats, density, threshold):
    """Reseat according to occupation density."""
    def reseat(occupied):
        d = density(occupied)
        return (seats & (d == 0)) | (occupied & (d <= threshold))
    return reseat

def neighbors(occupied):
    """Count the number of occupied neighboring seats."""
    occupied = np.pad(occupied, 1)
    return np.sum([occupied[s] for s in shifts], axis=0)

shifts = list(product([np.s_[:-2], np.s_[1:-1], np.s_[2:]], repeat=2))

def fixedpoint(f, x):
    while ((fx := f(x)) != x).any():
        x = fx
    return fx

seats = seatingplan == "L"
reseat = reseating(seats, density=neighbors, threshold=4)
assert 2344 == np.sum(fixedpoint(reseat, seats))

## Part 2

Array reversal/tranpose as functional transformations:

In [3]:
def rev(f):
    return lambda x: f(x[::-1])[::-1]

def transpose(f):
    return lambda x: f(x.T).T

For Part 2, occupation density is measured by seat visibility.

In [4]:
import numpy.ma as ma
import numba as nb

UNOCCUPIED = -np.inf

def visible(seats):
    """Count occupied seats that are in direct line of sight."""
    def lineofsight(occupied):
        occ = occupied.astype(float)
        occ[seats & ~occupied] = UNOCCUPIED
        return np.sum([vis(occ) & seats for vis in visibility], axis=0)
    return lineofsight

def visibleitself(occ):
    return occ == 1  # occupied seat is visible to itself

def visiblebelow(occ):
    """Is a point visible from below an occupied seat?"""
    occ = np.pad(occ[:-1], ((1, 0), (0, 0)))
    return reluadd.accumulate(occ).astype(bool)

@nb.vectorize([nb.float64(nb.float64, nb.float64)], nopython=True)
def reluadd(x, y):
    return np.maximum(0, x + y)

visibleabove = rev(visiblebelow)

def visiblealong(axis):
    def visibility(vis):
        """Is a point visible from below an occupied seat on axis?"""
        def visible(occ):
            d = axis(occ)
            return vis(d.data)[~d.mask].reshape(occ.shape)
        return visible
    return visibility

def antidiagonals(x):
    """Antidiagonals as (unmasked) columns."""
    m, n = x.shape
    i, j = np.indices((m, n))
    j = j + np.arange(m)[:, np.newaxis]
    diag = np.full((m, m + n - 1), UNOCCUPIED)
    diag[i, j] = x
    mask = np.ones(diag.shape, dtype=bool)
    mask[i, j] = False
    return ma.array(diag, mask=mask)

diagonally = visiblealong(rev(antidiagonals))
antidiagonally = visiblealong(antidiagonals)

visibility = (visibleitself,
              visiblebelow,
              visibleabove,
              transpose(visiblebelow),  # right
              transpose(visibleabove),  # left
              diagonally(visiblebelow),
              diagonally(visibleabove),
              antidiagonally(visiblebelow),
              antidiagonally(visibleabove))

reseat2 = reseating(seats, density=visible(seats), threshold=5)
assert 2076 == np.sum(fixedpoint(reseat2, seats))